In [ ]:
import pandas as pd
import numpy as np
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os
import sys
from sqlalchemy import create_engine
from importlib import reload


In [ ]:
import os, sys
from importlib import reload

MODULES_DIR = os.path.join(os.getcwd(), "..", "..", "modules")
sys.path.append(os.path.abspath(MODULES_DIR))

# 1. importe os módulos (não as funções)
import prefilter
import embedder
import help
import prompt_builder

# 2. recarregue apenas os módulos que você editou
reload(prefilter)
reload(embedder)
reload(help)
reload(prompt_builder)

# 3. agora sim importe as funções atualizadas
from prefilter import prefilter_results
from embedder import HuggingFaceEmbedder
from help import estimate_tokens
from prompt_builder import build_prompt

In [ ]:
load_dotenv()

# Pre-filter ddgo

### Get retrieval data

In [ ]:
USER = os.getenv("POSTGRES_USER")
PASSWORD = os.getenv("POSTGRES_PASSWORD")
DB = os.getenv("POSTGRES_DB")
PORT = os.getenv("POSTGRES_PORT")
HOST = os.getenv("POSTGRES_HOST", "localhost")

# Criar engine SQLAlchemy
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}")


In [ ]:
df = pd.read_sql("SELECT * FROM retrieved_news_ddgo", engine)

### Filter Data by suffle_id

In [ ]:
df_filt = df[df['shuffle_id'].between(0, 1999)]

In [ ]:
df_filt['search_title'].nunique()

In [ ]:
df_filt.groupby('shuffle_id').size().describe()

# Build Prompts

In [ ]:
def generate_prompts(filtered_by_title, mode='test1'):
    rows = []
    for title, filtered_results in filtered_by_title.items():

        shuffle_id = filtered_results[0].get("shuffle_id") if filtered_results else None

        prompt = build_prompt(
            mode=mode,
            title_to_check=title,
            results_filtered=filtered_results
        )

        info = {
            "search_title": title,
            "shuffle_id": shuffle_id,
            "prompt": prompt,
            "num_results": len(filtered_results),
            "approx_tokens": estimate_tokens(prompt),
            "prompt_length_chars": len(prompt),
        }
        rows.append(info)

    df_prompts = pd.DataFrame(rows)
    return df_prompts


In [ ]:
filtered_by_title = {i['search_title']: [{'shuffle_id': i['shuffle_id']}] for i in df_filt[['search_title', 'shuffle_id']].drop_duplicates().to_dict('records')}

In [ ]:
df_test4 = generate_prompts(filtered_by_title, mode="test5")
df_test4.to_sql(
    "test_tom_prompts",
    engine,
    if_exists="replace",   # substitui staging a cada execução
    index=False
)